In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from datetime import date
from yahooquery import Ticker
from utils_VaR import plot_scatter, plot_time_series_histogram, Security
from scipy.stats import ttest_ind
from statsmodels.stats.diagnostic import acorr_ljungbox

In [2]:
""" visualization """
# T: time horizon (in days) for computing risk metrics
# confidence level: 1-p for VaR, ES 
T = 120; p=5

t0='1920-01-01'
t1=date.today()

sp500 = Security('^GSPC')
sp500.set_df_hist(t0=t0, t1=t1)
sp500.set_df_pct_change()
sp500.set_df_risk(T=T, p=p, rolling_T=1) # to generate plot, set rolling T to 1. re-sampling is done in plot function for now
# read start and end date of past US recessions
recession_periods = pd.read_csv('../data/recession_periods_NBER.csv') # use dates classified by NBER
# generate high_vol_periods
sp500.label_vol_level_df(T=T)
df = sp500.get_df_risk()
df = df[df['vol'] > df['vol'].median()]
high_vol_periods = df[::-1][::T][::-1][['start', 'end']]
high_vol_periods.columns = ['Peak', 'Trough']

# scatter plot with mask
# plot_scatter(sp500.get_df_risk(), t0='1920-1-1', t1='2020-6-30', T=1, marked_periods=high_vol_periods)
# times series histogram
# plot_time_series_histogram(sp500.get_df_risk(), t0='2005-1-1', t1='2010-6-30', T=T, marked_periods=recession_periods, date_format='%B-%Y')

In [3]:
plot_scatter(sp500.get_df_risk(), t0='1920-1-1', t1='2020-6-30', T=1, marked_periods=high_vol_periods)

In [ ]:
plot_time_series_histogram(sp500.get_df_risk(), t0='2005-1-1', t1='2010-6-30', T=T, marked_periods=recession_periods, date_format='%B-%Y')

In [4]:
import pandas as pd 
import numpy as np
from utils_VaR import plot_scatter, plot_time_series_histogram, Security
from scipy.stats import ttest_ind
from datetime import date

""" collect analytic results for different assets """
# read recession periods
recession_periods = pd.read_csv('../data/recession_periods_NBER.csv') # use dates classified by NBER
recession_periods = recession_periods.astype({'Peak':'datetime64', 'Trough':'datetime64'})
peaks = recession_periods['Peak'].values
troughs = recession_periods['Trough'].values

# set asset ticker and set time horizon and p level for computing VaR, ES
securities = dict(sp500='^GSPC', agg='agg', gold='XAU=', oil='CLc1', GBPUSD='GBP=', tech_sec='.IXTTR', energy_sec='.IXE', russell_2k='.RUT') 
dict_Ts = dict(sp500=100, agg=100, gold=100, oil=100, GBPUSD=100, tech_sec=50, energy_sec=100, russell_2k=100)

# securities = dict(sp500='^GSPC', gold='XAU=') 
# dict_Ts = dict(sp500=100, gold=100)

p=5
# overall time horizon we want to consider
t0 = '1920-1-1'
t1 = date.today()

# compute VaR, ES for all securities
for i, key in enumerate(securities):
    # length of time period for computing VaR, ES
    T = dict_Ts[key]
    rolling_T = T
    # get historical data, compute VaR, ES
    securities[key] = Security(securities[key]) # input ticker
    securities[key].set_df_hist(t0=t0, t1=t1)
    securities[key].set_df_pct_change()
    securities[key].set_df_risk(T=dict_Ts[key], p=p, rolling_T=rolling_T)

In [18]:
""" by recession """
columns=['asset', 'VaR', 'ES', 'ES/VaR', 'recession','sample size', 't-statistic', 'p-value']
output = {}
for i, key in enumerate(securities):
    securities[key].label_recession_df(peaks=peaks, troughs=troughs, df='risk') # add recession (True/False) column
    df = securities[key].get_df_risk()
    
    r = df[df['recession'] == True][['VaR', 'ES']] # all VaR and ES in recession periods
    r_VaR_to_ESs = (r['ES']/r['VaR']).values # all VaR to ES ratios in recession periods
    nr = df[df['recession'] == False][['VaR', 'ES']]
    nr_VaR_to_ESs = (nr['ES']/nr['VaR']).values

    ttest_res = ttest_ind(a=r_VaR_to_ESs, b=nr_VaR_to_ESs, equal_var=False)

    output[i*2] = [key, np.mean(nr['VaR'].values), np.mean(nr['ES'].values), np.mean(nr_VaR_to_ESs), 'F', len(nr), '', '']
    output[i*2+1] = [key, np.mean(r['VaR'].values), np.mean(r['ES'].values), np.mean(r_VaR_to_ESs), 'T', len(r), ttest_res[0], ttest_res[1]]

df_tally = pd.DataFrame.from_dict(output, orient='index', columns=columns)
df_tally.to_csv(f'ES_VaR_Ratio_by_recession_{t0}-{t1}.csv', index=False)

In [5]:
""" by market vol levels """
columns=['asset', 'VaR', 'ES', 'ES/VaR', 'volatility level','sample size', 't-statistic', 'p-value']
output = {}
for i, key in enumerate(securities):

    securities[key].label_vol_level_df(T=dict_Ts[key]) # add vol level column

    df = securities[key].get_df_risk()
    
    low = df[df['vol level'] == 'low'][['VaR', 'ES']] 
    low_VaR_to_ESs = (low['ES']/low['VaR']).values 
    high = df[df['vol level'] == 'high'][['VaR', 'ES']]
    high_VaR_to_ESs = (high['ES']/high['VaR']).values

    ttest_res = ttest_ind(a=low_VaR_to_ESs, b=high_VaR_to_ESs, equal_var=False)

    output[i*2] = [key, np.mean(low['VaR'].values), np.mean(low['ES'].values), np.mean(low_VaR_to_ESs), 'Low', len(low), '', '']
    output[i*2+1] = [key, np.mean(high['VaR'].values), np.mean(high['ES'].values), np.mean(high_VaR_to_ESs), 'High', len(high), ttest_res[0], ttest_res[1]]

df_tally = pd.DataFrame.from_dict(output, orient='index', columns=columns)
df_tally.to_csv(f'ES_VaR_Ratio_by_realized_vol_{t0}-{t1}.csv', index=False)

Taking non-overlapping time periods leaves us with fewer data points but it's pretty much in keeping with the assumption that
two samples are independent when doing a t-test. However, we are left with small sample size (<30), so that the mean of a sample should follow a normal distribution may not hold. 

Taking over-lapping time periods significant violated t-test assumption due to strong auto-correlations. Currently, historical prices for all securities except sp500 are only available from 2000 to today, on Yahoo finance. For some securities, such as GBPUSD, more data seems to be available elsewhere, which can be 'manually' fed into the program. 

In [21]:
""" Generate Results for Random LS Portfolios on sp500 constituents """

from datetime import date
from utils_VaR import Security
from scipy.stats import ttest_ind
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


t0 = pd.to_datetime('1985-1-1')
t1 = date.today()

# sp500_const = pd.read_csv('../data/sp500_const.csv', parse_dates=[0], index_col=0).loc[t0:].dropna(axis='columns')
# tickers = sp500_const.columns
# remove ones that were not included in sp500 at the time
df = pd.read_csv('../data/sp500_historical_constituents.csv').astype({'from':'datetime64'})
elig_tickers = df[(df['from']>t0) & (df['thru'].isnull())]['co_tic'].values

In [22]:
from yahooquery import Ticker
date_index = pd.to_datetime(pd.read_csv('../data/business_date.csv', header=None)[0].values)
sp500_const_new = pd.DataFrame(index=date_index)
for ticker in elig_tickers:
    try:
        series = Ticker(ticker).history(period='max')['adjclose']
        series.index = series.index.normalize()
        sp500_const_new[ticker] = series
    except:
        print(ticker)

BRK.B


In [23]:
# elig_tickers = list(map(str,elig_tickers))
# Ticker('APD').history(period='max')['adjclose']
sp500_const = sp500_const_new.loc[t0:].dropna(axis='columns')
tickers = sp500_const.columns

In [24]:
np.random.seed(1)

num_iter = 2000
portfolios = {}
T = 100
p = 5
num_const =20
average_pct_change = np.array([0.]*(len(sp500_const)-1))
for i in range(num_iter):
    sampled_tickers = [tickers[i] for i in np.random.choice(len(tickers), num_const)] # sample from tickers universe
    weights = np.random.permutation([0.1]*10+[-0.1]*10)

    # computes portfolio pct_change
    pct_change = sp500_const[sampled_tickers].pct_change().dropna().values @ weights
    average_pct_change += pct_change
    portfolio_df = pd.DataFrame(index=sp500_const.index[1:], data=pct_change)

    # save result from each iteration
    portfolios[i] = Security(i)
    portfolios[i].set_df_pct_change_local(t0=t0, t1=t1, df=portfolio_df)
    portfolios[i].set_df_risk(T, p)

In [25]:
""" by recession """

# read recession periods
recession_periods = pd.read_csv('../data/recession_periods_NBER.csv') # use dates classified by NBER
recession_periods = recession_periods.astype({'Peak':'datetime64', 'Trough':'datetime64'})
peaks = recession_periods['Peak'].values
troughs = recession_periods['Trough'].values

columns=['asset', 'VaR', 'ES', 'ES/VaR', 'recession']
output = {}
for i, key in enumerate(portfolios):
    portfolios[key].label_recession_df(peaks=peaks, troughs=troughs, df='risk') # add recession (True/False) column
    df = portfolios[key].get_df_risk()
    
    r = df[df['recession'] == True][['VaR', 'ES']] # all VaR and ES in recession periods
    r_VaR_to_ESs = (r['ES']/r['VaR']).values # all VaR to ES ratios in recession periods
    nr = df[df['recession'] == False][['VaR', 'ES']]
    nr_VaR_to_ESs = (nr['ES']/nr['VaR']).values

    # ttest_res = ttest_ind(a=r_VaR_to_ESs, b=nr_VaR_to_ESs, equal_var=False)

    output[i*2] = [key, np.mean(nr['VaR'].values), np.mean(nr['ES'].values), np.mean(nr_VaR_to_ESs), 'F']
    output[i*2+1] = [key, np.mean(r['VaR'].values), np.mean(r['ES'].values), np.mean(r_VaR_to_ESs), 'T']


temp= pd.DataFrame.from_dict(output, orient='index', columns=columns)

nr_VaR_bar = np.mean(temp[temp['recession'] == 'F']['VaR'].values)
r_VaR_bar = np.mean(temp[temp['recession'] == 'T']['VaR'].values)
nr_ES_bar = np.mean(temp[temp['recession'] == 'F']['ES'].values)
r_ES_bar = np.mean(temp[temp['recession'] == 'T']['ES'].values)
nr_ratios = temp[temp['recession'] == 'F']['ES/VaR'].values
r_ratios = temp[temp['recession'] == 'T']['ES/VaR'].values

ttest_res = ttest_ind(a=r_ratios, b=nr_ratios, equal_var=False)

pd.DataFrame.from_dict({0: [nr_VaR_bar, nr_ES_bar, np.mean(nr_ratios), 'F', len(nr)*num_iter,'', ''], 1:[r_VaR_bar, r_ES_bar, np.mean(r_ratios), 'T', len(r)*num_iter ,ttest_res[0], ttest_res[1]]} , columns=['VaR', 'ES', 'ES/VaR', 'recession','sample size', 't-statistics', 'p-value'], orient='index')

,VaR,ES,ES/VaR,recession,sample size,t-statistics,p-value
0,0.012094,0.016712,1.385134,F,156000,,
1,0.017078,0.023319,1.370453,T,22000,-9.13155,1.15931e-19


In [26]:
""" by market vol levels """
columns=['asset', 'VaR', 'ES', 'ES/VaR', 'vol level']
output = {}
for i, key in enumerate(portfolios):
    portfolios[key].label_vol_level_df(T=T) # add vol level column
    df = portfolios[key].get_df_risk()
    
    low = df[df['vol level'] == 'low'][['VaR', 'ES']] 
    low_VaR_to_ESs = (low['ES']/low['VaR']).values 
    high = df[df['vol level'] == 'high'][['VaR', 'ES']]
    high_VaR_to_ESs = (high['ES']/high['VaR']).values

    # ttest_res = ttest_ind(a=low_VaR_to_ESs, b=high_VaR_to_ESs, equal_var=False)

    output[i*2] = [key, np.mean(low['VaR'].values), np.mean(low['ES'].values), np.mean(low_VaR_to_ESs), 'Low']
    output[i*2+1] = [key, np.mean(high['VaR'].values), np.mean(high['ES'].values), np.mean(high_VaR_to_ESs), 'High']

temp= pd.DataFrame.from_dict(output, orient='index', columns=columns)

low_VaR_bar = np.mean(temp[temp['vol level'] == 'Low']['VaR'].values)
high_VaR_bar = np.mean(temp[temp['vol level'] == 'High']['VaR'].values)
low_ES_bar = np.mean(temp[temp['vol level'] == 'Low']['ES'].values)
high_ES_bar = np.mean(temp[temp['vol level'] == 'High']['ES'].values)
low_ratios = temp[temp['vol level'] == 'Low']['ES/VaR'].values
high_ratios = temp[temp['vol level'] == 'High']['ES/VaR'].values

ttest_res = ttest_ind(a=low_ratios, b=high_ratios, equal_var=False)

pd.DataFrame.from_dict({0: [low_VaR_bar, low_ES_bar, np.mean(low_ratios), 'Low', len(low)*num_iter,'', ''], 1:[high_VaR_bar, high_ES_bar, np.mean(high_ratios), 'High', len(high)*num_iter ,ttest_res[0], ttest_res[1]]} , columns=['VaR', 'ES', 'ES/VaR', 'vol level','sample size', 't-statistics', 'p-value'], orient='index')

,VaR,ES,ES/VaR,vol level,sample size,t-statistics,p-value
0,0.009471,0.012794,1.362492,Low,90000,,
1,0.016022,0.022371,1.404620,High,88000,-28.6868,1.73946e-160


In [ ]:
""" generate sharpe ratio for insanity check """

columns=['asset', 'annualized return', 'annualized std', 'Sharpe Ratio']
output = {}

for i, key in enumerate(portfolios):
    df = portfolios[key].get_df_pct_change()
    annual_return = (df+1).cumprod().values[-1][0] **(1/(t1.year-t0.year)) - 1
    annual_std = df.std().values[0] * ((252)**0.5)

    output[i] = [key, annual_return, annual_std, annual_return/annual_std]

df_tally = pd.DataFrame.from_dict(output, orient='index', columns=columns)

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_tally)

In [1]:
""" fisher's combined p-value test on random LS t-statistics"""

from datetime import date
from utils_VaR import Security
from scipy.stats import ttest_ind
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# read tickers
t0 = pd.to_datetime('1985-1-1')
t1 = date.today()
# remove ones that were not included in sp500 at the time
df = pd.read_csv('../data/sp500_historical_constituents.csv').astype({'from':'datetime64'})
elig_tickers = df[(df['from']>t0) & (df['thru'].isnull())]['co_tic'].values

from yahooquery import Ticker
date_index = pd.to_datetime(pd.read_csv('../data/business_date.csv', header=None)[0].values)
sp500_const_new = pd.DataFrame(index=date_index)
for ticker in elig_tickers:
    try:
        series = Ticker(ticker).history(period='max')['adjclose']
        series.index = series.index.normalize()
        sp500_const_new[ticker] = series
    except:
        print(ticker)

BRK.B


In [2]:
# only keep stocks with price data available since t0
# generate tickers that meet requirements (i.e. stock price available after t0)
sp500_const = sp500_const_new.loc[t0:].dropna(axis='columns')
tickers = sp500_const.columns

In [5]:
import csv
with open('tickers.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile)
     wr.writerow(tickers)

In [50]:
# sample stocks with replacement to form random LS portfolio
np.random.seed(1)
num_iter = 15
portfolios = {}
ttest_results = list()
T = 100
p = 5
num_const =20

for i in range(num_iter):
    # randomly choose tickers
    sampled_tickers = [tickers[i] for i in np.random.choice(len(tickers), num_const)] # sample from tickers universe
    # randomly assign weights to form portfolio
    weights = np.random.permutation([0.1]*10+[-0.1]*10)
    # computes portfolio pct_change
    pct_change = sp500_const[sampled_tickers].pct_change().dropna().values @ weights
    average_pct_change += pct_change
    portfolio_df = pd.DataFrame(index=sp500_const.index[1:], data=pct_change)
    # save result from each iteration
    portfolios[i] = Security(i)
    portfolios[i].set_df_pct_change_local(t0=t0, t1=t1, df=portfolio_df)
    portfolios[i].set_df_risk(T, p)
    portfolios[i].label_vol_level_df(T=T) # add vol level column

    df = portfolios[i].get_df_risk()
    low = df[df['vol level'] == 'low'][['VaR', 'ES']] 
    low_VaR_to_ESs = (low['ES']/low['VaR']).values 
    high = df[df['vol level'] == 'high'][['VaR', 'ES']]
    high_VaR_to_ESs = (high['ES']/high['VaR']).values

    ttest_res = ttest_ind(a=low_VaR_to_ESs, b=high_VaR_to_ESs, equal_var=False)
    ttest_results.append(ttest_res)

In [53]:
ttest_results

[Ttest_indResult(statistic=-0.5745233332867739, pvalue=0.5675411769234175),
 Ttest_indResult(statistic=-2.1516630926094966, pvalue=0.035109899226506),
 Ttest_indResult(statistic=-0.9902424851445796, pvalue=0.32481906917399817),
 Ttest_indResult(statistic=-0.9301415688834328, pvalue=0.3558654857571747),
 Ttest_indResult(statistic=1.004116793034634, pvalue=0.3181545735417874),
 Ttest_indResult(statistic=-0.9462746385084716, pvalue=0.3474456574898278),
 Ttest_indResult(statistic=2.5001591442697175, pvalue=0.014484512989350542),
 Ttest_indResult(statistic=-0.6030641942863405, pvalue=0.5482608337498884),
 Ttest_indResult(statistic=-2.2013826408811212, pvalue=0.03064626481404009),
 Ttest_indResult(statistic=0.6348938554553846, pvalue=0.5272063184798575),
 Ttest_indResult(statistic=-1.6013171380172397, pvalue=0.11333504888475544),
 Ttest_indResult(statistic=-1.941501262551403, pvalue=0.057593481328996075),
 Ttest_indResult(statistic=-1.3537221186413497, pvalue=0.18013363867457158),
 Ttest_ind

In [51]:
# for ttest in ttest_results:
#     print(ttest)

# by order of paramter input (low vol first, then high vol), if t-statistics is negative, then ES/VaR ratio is higher in high vol periods. 
ttests = np.array([t[0] for t in ttest_results])
print(f"proportion of test w/ negative t-statistics (ES/VaR higher in high vol periods): {sum(ttests<0)/len(ttest_results)}")
ttests_sig = np.array([t[0] for t in ttest_results if t[1]<0.05])
print(f"proportion of test w/ negative t-statistics and statistically significant p-value: {sum(ttests_sig<0)/len(ttest_results)}")
print(f"proportion of test w/ positive t-statistics and statistically significant p-value: {sum(ttests_sig>0)/len(ttest_results)}")
pvalues = np.array([t[1] for t in ttest_results])
print(f"proportion of test w/ statistically significant p-value: {sum(pvalues<0.05)/len(ttest_results)}")
print('out of those tests the conclude ES/VaR ratios are different in different market vol periods, absolute majority of indicate ES/VaR is higher in high market vol periods')

proportion of test w/ negative t-statistics (ES/VaR higher in high vol periods): 0.8
proportion of test w/ negative t-statistics and statistically significant p-value: 0.2
proportion of test w/ positive t-statistics and statistically significant p-value: 0.06666666666666667
proportion of test w/ statistically significant p-value: 0.26666666666666666
out of those tests the conclude ES/VaR ratios are different in different market vol periods, absolute majority of indicate ES/VaR is higher in high market vol periods


In [52]:
from scipy.stats import combine_pvalues
combine_pvalues(pvalues, method='fisher')

(65.27042879613583, 0.0002025311348970786)

In [59]:
# sample stocks/tickers without replacement to form Random LS portfolio
np.random.seed(1)
num_iter = 10
portfolios = {}
ttest_results = list()
T = 100
p = 5
num_const =20
tickers_wo_replace = tickers.values
np.random.shuffle(tickers_wo_replace)
tickers_wo_replace = [tickers_wo_replace[i:i + num_const] for i in range(0, len(tickers_wo_replace), num_const)]

for i in range(num_iter):
    # randomly choose tickers
    sampled_tickers = tickers_wo_replace[i] # sample from tickers universe
    if len(sampled_tickers) != num_const:
        break
    # randomly assign weights to form portfolio
    weights = np.random.permutation([0.1]*10+[-0.1]*10)
    # computes portfolio pct_change
    pct_change = sp500_const[sampled_tickers].pct_change().dropna().values @ weights
    average_pct_change += pct_change
    portfolio_df = pd.DataFrame(index=sp500_const.index[1:], data=pct_change)
    # save result from each iteration
    portfolios[i] = Security(i)
    portfolios[i].set_df_pct_change_local(t0=t0, t1=t1, df=portfolio_df)
    portfolios[i].set_df_risk(T, p)
    portfolios[i].label_vol_level_df(T=T) # add vol level column

    df = portfolios[i].get_df_risk()
    low = df[df['vol level'] == 'low'][['VaR', 'ES']] 
    low_VaR_to_ESs = (low['ES']/low['VaR']).values 
    high = df[df['vol level'] == 'high'][['VaR', 'ES']]
    high_VaR_to_ESs = (high['ES']/high['VaR']).values

    ttest_res = ttest_ind(a=low_VaR_to_ESs, b=high_VaR_to_ESs, equal_var=False)
    ttest_results.append(ttest_res)

In [60]:
print('t-test for random LS portfolios with no overlapping constituents')
ttest_results

t-test for random LS portfolios with no overlapping constituents


[Ttest_indResult(statistic=1.4838341259052303, pvalue=0.1415697069813103),
 Ttest_indResult(statistic=-0.6068911578043475, pvalue=0.5456727496178501),
 Ttest_indResult(statistic=-0.9026287803414046, pvalue=0.3692516532542286),
 Ttest_indResult(statistic=-1.5556744015482429, pvalue=0.12441680412175765),
 Ttest_indResult(statistic=-1.880038893234535, pvalue=0.06500697289405814)]

In [62]:
combine_pvalues([t[1] for t in ttest_results], method='fisher')

(16.748709086378096, 0.08011265358953185)